<a href="https://colab.research.google.com/github/rajivsresearch/SP500/blob/master/Compare_Investment_at_10_percent_down.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

     |████████████████████████████████| 5.5MB 6.1MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=1f33d05262bdd701b6d107174560903495264dfb6c0e9745110fabea27a67f23
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.express as px

In [3]:
def GetStockData(tickerSymbol, start, end):
  #get data on this ticker
  tickerData = yf.Ticker(tickerSymbol)

  #get the historical prices for this ticker
  tickerDf = tickerData.history(period='1d', start=start, end=end)

  #see your data
  return tickerDf[['Close']]

In [4]:
spy = GetStockData('SPY', '2000-01-01', '2020-12-31')

In [5]:
spy.head()

,Close
Date,
2000-01-03,98.146072
2000-01-04,94.307983
2000-01-05,94.476685
2000-01-06,92.958336
2000-01-07,98.356987


In [6]:
years = np.arange(2000,2021,1)

In [7]:
years

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [8]:
spunits = pd.DataFrame(index=years)

In [9]:
currprice = spy.iloc[-1].values[0]

In [10]:
spunits['Cash'] = np.arange(1000,22000,1000)

Calculate returns for investing on the first day of the year 

In [11]:
units = 0
totunits = 0
purch = 0
totvalu = 0
price = 0

for y in years:
  f1 = ((spy.index.year == y ) & (spy.index.month == 1))
  price = spy.loc[f1, 'Close'].head(1).values[0]
  units = (1000/price)
  purch = purch + 1
  totunits = totunits + units
  totvalu = totunits*price
  print('Units Purchased in year ', y, ' = ', units, '. Total Portfolio = ', totvalu)
  spunits.loc[y,'Portfolio Value Start'] = totvalu

print('Units Purchased in 20 years = ', totunits, '. Total Value = ', totunits*currprice)
print('Total Book Value = ', purch*1000+(21-purch)*1000)
spunits.loc[2020,'Portfolio Value Start'] = totunits*currprice

Units Purchased in year  2000  =  10.188894732840794 . Total Portfolio =  1000.0
Units Purchased in year  2001  =  11.384185334870773 . Total Portfolio =  1895.0042917547469
Units Purchased in year  2002  =  12.532745325439555 . Total Portfolio =  2721.3371458144543
Units Purchased in year  2003  =  15.650723499088727 . Total Portfolio =  3179.1852239378554
Units Purchased in year  2004  =  12.608492513082302 . Total Portfolio =  4946.272628596442
Units Purchased in year  2005  =  11.438820567116672 . Total Portfolio =  6452.051725035687
Units Purchased in year  2006  =  10.672783888096017 . Total Portfolio =  7915.1462960621375
Units Purchased in year  2007  =  9.391590158892933 . Total Portfolio =  9994.924654004793
Units Purchased in year  2008  =  8.994911059344439 . Total Portfolio =  11435.704744619123
Units Purchased in year  2009  =  13.693978774082156 . Total Portfolio =  8511.56028322869
Units Purchased in year  2010  =  10.978464713573404 . Total Portfolio =  11616.887597110

In [48]:
df = spy.loc[spy.index.year==2020,:].reset_index()

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    248 non-null    datetime64[ns]
 1   Close   248 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 4.0 KB


In [28]:
df.head()

,Date,Close
0,2020-01-02,318.914307
1,2020-01-03,316.499451
2,2020-01-06,317.706909
3,2020-01-07,316.813568
4,2020-01-08,318.502075


In [43]:
def getreturns(df, buydate):
  f1 = df['Date'] == buydate
  buyprice = df.loc[f1, 'Close'].values

  f2 = df['Date'] > buydate
  return (df.loc[f2,'Close'] - buyprice)/buyprice*100

In [50]:
df.head()

,Date,Close,2020-01-02 00:00:00,2020-01-03 00:00:00,2020-01-06 00:00:00,2020-01-07 00:00:00,2020-01-08 00:00:00,2020-01-09 00:00:00,2020-01-10 00:00:00,2020-01-13 00:00:00,2020-01-14 00:00:00,2020-01-15 00:00:00,2020-01-16 00:00:00,2020-01-17 00:00:00,2020-01-21 00:00:00,2020-01-22 00:00:00,2020-01-23 00:00:00,2020-01-24 00:00:00,2020-01-27 00:00:00,2020-01-28 00:00:00,2020-01-29 00:00:00,2020-01-30 00:00:00,2020-01-31 00:00:00,2020-02-03 00:00:00,2020-02-04 00:00:00,2020-02-05 00:00:00,2020-02-06 00:00:00,2020-02-07 00:00:00,2020-02-10 00:00:00,2020-02-11 00:00:00,2020-02-12 00:00:00,2020-02-13 00:00:00,2020-02-14 00:00:00,2020-02-18 00:00:00,2020-02-19 00:00:00,2020-02-20 00:00:00,2020-02-21 00:00:00,2020-02-24 00:00:00,2020-02-25 00:00:00,2020-02-26 00:00:00,...,2020-10-28 00:00:00,2020-10-29 00:00:00,2020-10-30 00:00:00,2020-11-02 00:00:00,2020-11-03 00:00:00,2020-11-04 00:00:00,2020-11-05 00:00:00,2020-11-06 00:00:00,2020-11-09 00:00:00,2020-11-10 00:00:00,2020-11-11 00:00:00,2020-11-12 00:00:00,2020-11-13 00:00:00,2020-11-16 00:00:00,2020-11-17 00:00:00,2020-11-18 00:00:00,2020-11-19 00:00:00,2020-11-20 00:00:00,2020-11-23 00:00:00,2020-11-24 00:00:00,2020-11-25 00:00:00,2020-11-27 00:00:00,2020-11-30 00:00:00,2020-12-01 00:00:00,2020-12-02 00:00:00,2020-12-03 00:00:00,2020-12-04 00:00:00,2020-12-07 00:00:00,2020-12-08 00:00:00,2020-12-09 00:00:00,2020-12-10 00:00:00,2020-12-11 00:00:00,2020-12-14 00:00:00,2020-12-15 00:00:00,2020-12-16 00:00:00,2020-12-17 00:00:00,2020-12-18 00:00:00,2020-12-21 00:00:00,2020-12-22 00:00:00,2020-12-23 00:00:00
0,2020-01-02,318.914307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-03,316.499451,-0.757212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-06,317.706909,-0.378596,0.381504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-07,316.813568,-0.658716,0.099247,-0.281184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-08,318.502075,-0.129261,0.632742,0.250283,0.532966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
units = 0
totunits = 0
funds = 1000
purch = 0
totvalu = 0
strprice = 0
purch = 0

for year in years:
  print(year)
  df = spy.loc[spy.index.year==year,:].reset_index()

  for date in df['Date']:
    df[date] = getreturns(date)

  if df.loc[df.iloc[:,2]<=-10].isnull().values.any():
    strprice = df.loc[df.iloc[:,2]<=-10].iloc[0,1]
    print('Buy Price = ', strprice)
    print('Buy Date = ', df.loc[df.iloc[:,2]<=-10].iloc[0,0])

    units = funds/strprice
    purch = purch + funds/1000
    totunits = totunits + units
    totvalu = totunits * strprice
    funds = 1000

    print('Units Purchased in year ', year, ' = ', units, '. Total Portfolio value = ', totvalu)
    spunits.loc[year,'Portfolio Value 10 pct'] = totvalu
  else:
    funds = funds + 1000
    spunits.loc[year,'Portfolio Value 10 pct'] = totvalu + funds - 1000

print('Units Purchased in 20 years = ', totunits, '. Total Portfolio Value = ', totunits*currprice)
print('Total Book Value = ', purch*1000)
spunits.loc[2020,'Portfolio Value 10 pct'] = totunits*currprice

2000
Buy Price =  86.09371185302734
Buy Date =  2000-12-20 00:00:00
Units Purchased in year  2000  =  11.615250155634179 . Total Portfolio value =  1000.0
2001
Buy Price =  78.63999938964844
Buy Date =  2001-03-16 00:00:00
Units Purchased in year  2001  =  12.716175073262173 . Total Portfolio value =  1913.4232651496857
2002
Buy Price =  71.65950012207031
Buy Date =  2002-06-06 00:00:00
Units Purchased in year  2002  =  13.954883836707246 . Total Portfolio value =  2743.5777691602425
2003
Buy Price =  57.054203033447266
Buy Date =  2003-03-10 00:00:00
Units Purchased in year  2003  =  17.527192508740562 . Total Portfolio value =  3184.39485083026
2004
2005
2006
2007
2008
Buy Price =  99.49887084960938
Buy Date =  2008-03-07 00:00:00
Units Purchased in year  2008  =  50.251826551453064 . Total Portfolio value =  10553.380384810138
2009
Buy Price =  63.29182815551758
Buy Date =  2009-01-20 00:00:00
Units Purchased in year  2009  =  15.799828021128556 . Total Portfolio value =  7713.06852

In [97]:
spunits

,Cash,Portfolio Value Start,Portfolio Value 10 pct
2000,1000,1000.000000,1000.000000
2001,2000,1895.004292,1913.423265
2002,3000,2721.337146,2743.577769
2003,4000,3179.185224,3184.394851
2004,5000,4946.272629,4184.394851
2005,6000,6452.051725,5184.394851
2006,7000,7915.146296,6184.394851
2007,8000,9994.924654,7184.394851
2008,9000,11435.704745,10553.380385
2009,10000,8511.560283,7713.068521


In [99]:
df = spunits.reset_index().melt(id_vars=['index'])
df.columns = ['Year', 'Type of Investment', 'Portfolio Value']
df.head()

,Year,Type of Investment,Portfolio Value
0,2000,Cash,1000.0
1,2001,Cash,2000.0
2,2002,Cash,3000.0
3,2003,Cash,4000.0
4,2004,Cash,5000.0


In [100]:
fig = px.bar(df, x='Type of Investment', y='Portfolio Value', animation_frame="Year", animation_group = 'Type of Investment', 
             range_y=[0,df['Portfolio Value'].max()*1.1])
fig.show()